## 🛠️ Scikit-Learn Pipelines 

### 📌 What is a Pipeline?
A pipeline in scikit-learn is a way to chain multiple preprocessing steps and a final estimator into one complete workflow. It ensures clean and reproducible modeling without manual handling at each step.

### 🎯 Why Use a Pipeline?
- Keeps code organized and readable

- Prevents data leakage (no peeking into test data)

- Allows consistent preprocessing in both training and testing

- Makes hyperparameter tuning easier with tools like GridSearchCV

- Ensures reproducibility



In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [5]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# plan
# there are values missing in two columns: age and embarked
# we will use SimpleImputer to fill the missing values with column transformer
# then the output of this will be passed to OneHotEncoder for categorical columns with column transformer
# then the output of this will be passed to MinMaxScaler for numerical columns with column transformer
# then the output of this will be passed to SelectKBest for feature selection with column transformer
# then the output of this will be passed to DecisionTreeClassifier for classification 

In [7]:
# usually if you are calling any column, call it by number not by name

In [ ]:
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

In [10]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [11]:
df['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [13]:
y_train.sample(3)

315    1
490    0
259    1
Name: Survived, dtype: int64

In [33]:
# imputation for missing values
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [35]:
# one-hot encoding for categorical columns
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [50]:
# scaling for numerical columns
trf3 = ColumnTransformer(
    transformers=[
('scale', MinMaxScaler(), slice(0, 10))
    ]
)

In [51]:
# feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [52]:
# training the model
trf5 = DecisionTreeClassifier()

#### Now we will create the pipeline

In [53]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)

])

In [54]:
# Pipeline vs make_pipeline
# Pipeline allows you to name each step, which is useful for debugging and understanding the flow of transformations.
# make_pipeline automatically names the steps based on the class names of the transformers, which can be convenient for quick setups.
# In practice, you can use either based on your preference and the complexity of your pipeline.
# (same applies to ColumnTransformer vs make_column_transformer)

In [55]:
pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [62]:
pipe.fit(X_train, y_train)

,steps,"[('columntransformer-1', ...), ('columntransformer-2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('impute_age', ...), ('impute_embarked', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [60]:
# explore the pipeline
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x000001EC657428E0>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [63]:
# predict
y_pred = pipe.predict(X_test)

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6256983240223464

#### Cross Validation using Pipeline

In [65]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

np.float64(0.6391214419383433)

#### GridSearch using Pipeline

In [95]:
params = {
    'decisiontreeclassifier__max_depth': [3, 5, None],
    'decisiontreeclassifier__min_samples_split': [2, 5, 10]
}


In [96]:
print(pipe.named_steps)

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])]), 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 6])]), 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]), 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x000001EC657428E0>), 'decisiontreeclassifier': DecisionTreeClassifier()}


In [97]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)


,estimator,Pipeline(step...lassifier())])
,param_grid,"{'decisiontreeclassifier__max_depth': [3, 5, ...], 'decisiontreeclassifier__min_samples_split': [2, 5, ...]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('impute_age', ...), ('impute_embarked', ...)]"


#### Exporting the Pipeline

In [98]:
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))